In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models, regularizers
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import os

# Set a random seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Data Generation Function for each person
def generate_eye_tracking_data(num_samples):
    # Generate synthetic eye-tracking metrics
    fixation_duration = np.random.uniform(150, 600, num_samples)  # in milliseconds
    saccade_amplitude = np.random.uniform(1, 40, num_samples)     # in degrees
    pupil_dilation = np.random.uniform(2, 8, num_samples)         # in millimeters
    blink_rate = np.random.uniform(10, 20, num_samples)           # blinks per minute
    gaze_x = np.random.uniform(0, 1920, num_samples)              # screen resolution width
    gaze_y = np.random.uniform(0, 1080, num_samples)              # screen resolution height

    # Combine features into a single dataset
    X = np.column_stack((fixation_duration, saccade_amplitude, pupil_dilation, blink_rate, gaze_x, gaze_y))

    # Generate binary labels (0: Healthy, 1: Early-stage Alzheimer's)
    # Higher fixation duration, pupil dilation, and blink rate may indicate Alzheimer's.
    y = (fixation_duration + pupil_dilation + blink_rate) > (fixation_duration.mean() + pupil_dilation.mean() + blink_rate.mean())
    y = y.astype(int)

    return X, y

# Function to generate and save data for multiple individuals
def generate_and_save_data(num_individuals, num_samples_per_person, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    all_data = []

    for i in range(num_individuals):
        X, y = generate_eye_tracking_data(num_samples_per_person)
        person_data = np.column_stack((np.full(num_samples_per_person, i), X, y))
        all_data.append(person_data)

        # Save each person's data to a CSV file
        df = pd.DataFrame(person_data, columns=['PersonID', 'FixationDuration', 'SaccadeAmplitude', 'PupilDilation', 'BlinkRate', 'GazeX', 'GazeY', 'Label'])
        df.to_csv(os.path.join(output_dir, f'person_{i}.csv'), index=False)

    # Combine all data into a single dataframe
    all_data = np.vstack(all_data)
    df_all = pd.DataFrame(all_data, columns=['PersonID', 'FixationDuration', 'SaccadeAmplitude', 'PupilDilation', 'BlinkRate', 'GazeX', 'GazeY', 'Label'])
    df_all.to_csv(os.path.join(output_dir, 'all_data.csv'), index=False)

# Generate and save data
output_dir = 'eye_tracking_data'
num_individuals = 400  # Number of individuals
num_samples_per_person = 4  # Number of samples per individual

generate_and_save_data(num_individuals, num_samples_per_person, output_dir)

# Load the data from CSV
df = pd.read_csv(os.path.join(output_dir, 'all_data.csv'))

# Split the data into features and labels
X = df[['FixationDuration', 'SaccadeAmplitude', 'PupilDilation', 'BlinkRate', 'GazeX', 'GazeY']].values
y = df['Label'].values

# Reshape X for LSTM input [samples, timesteps, features]
X = X.reshape((X.shape[0], 1, X.shape[1]))

# Split the data into 70% train, 15% validation, 15% test
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Build the Bi-LSTM Model with Attention Mechanism
def build_bilstm_attention_model(input_shape):
    inputs = layers.Input(shape=input_shape)

    # Bi-LSTM Layer
    x = layers.Bidirectional(layers.LSTM(128, return_sequences=True))(inputs)

    # Attention Mechanism
    attention = layers.Attention()([x, x])

    # Flatten the output of the attention layer
    x = layers.Flatten()(attention)

    # Dense Layers
    x = layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.001))(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(32, activation='relu', kernel_regularizer=regularizers.l2(0.001))(x)
    x = layers.Dropout(0.5)(x)

    # Output Layer
    outputs = layers.Dense(1, activation='sigmoid')(x)

    model = models.Model(inputs, outputs)
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    return model

# Build and Train the Model
model = build_bilstm_attention_model((X_train.shape[1], X_train.shape[2]))
history = model.fit(X_train, y_train,
                    epochs=50,
                    batch_size=32,
                    validation_data=(X_val, y_val),
                    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)])

# Evaluate the Model
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {test_acc:.4f}')

# Plot the Training and Validation Loss
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

# Plot the Training and Validation Accuracy
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models, regularizers
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import os

# Set a random seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Data Generation Function
def generate_eye_tracking_data(num_samples, category):
    # Generate synthetic eye-tracking metrics with different distributions based on the category
    if category == 'healthy':
        fixation_duration = np.random.uniform(150, 300, num_samples)  # Healthy range
        saccade_amplitude = np.random.uniform(20, 40, num_samples)    # Healthy range
        pupil_dilation = np.random.uniform(2, 4, num_samples)         # Healthy range
        blink_rate = np.random.uniform(10, 15, num_samples)           # Healthy range
    elif category == 'early_alz':
        fixation_duration = np.random.uniform(300, 450, num_samples)  # Early-stage range
        saccade_amplitude = np.random.uniform(10, 20, num_samples)    # Early-stage range
        pupil_dilation = np.random.uniform(4, 6, num_samples)         # Early-stage range
        blink_rate = np.random.uniform(15, 20, num_samples)           # Early-stage range
    else:  # category == 'alz'
        fixation_duration = np.random.uniform(450, 600, num_samples)  # Alzheimer's range
        saccade_amplitude = np.random.uniform(1, 10, num_samples)     # Alzheimer's range
        pupil_dilation = np.random.uniform(6, 8, num_samples)         # Alzheimer's range
        blink_rate = np.random.uniform(20, 25, num_samples)           # Alzheimer's range

    gaze_x = np.random.uniform(0, 1920, num_samples)              # screen resolution width
    gaze_y = np.random.uniform(0, 1080, num_samples)              # screen resolution height

    # Combine features into a single dataset
    X = np.column_stack((fixation_duration, saccade_amplitude, pupil_dilation, blink_rate, gaze_x, gaze_y))

    # Generate labels (0: Healthy, 1: Early-stage Alzheimer's, 2: Alzheimer's)
    y = 0 if category == 'healthy' else (1 if category == 'early_alz' else 2)

    return X, np.full(num_samples, y)

# Function to generate and save data for multiple individuals
def generate_and_save_data(num_individuals, num_samples_per_person, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    all_data = []

    for i in range(num_individuals):
        if i < 200:
            category = 'healthy'
        elif i < 300:
            category = 'early_alz'
        else:
            category = 'alz'

        X, y = generate_eye_tracking_data(num_samples_per_person, category)
        person_data = np.column_stack((np.full(num_samples_per_person, i), X, y))
        all_data.append(person_data)

        # Save each person's data to a CSV file
        df = pd.DataFrame(person_data, columns=['PersonID', 'FixationDuration', 'SaccadeAmplitude', 'PupilDilation', 'BlinkRate', 'GazeX', 'GazeY', 'Label'])
        df.to_csv(os.path.join(output_dir, f'person_{i}.csv'), index=False)

    # Combine all data into a single dataframe
    all_data = np.vstack(all_data)
    df_all = pd.DataFrame(all_data, columns=['PersonID', 'FixationDuration', 'SaccadeAmplitude', 'PupilDilation', 'BlinkRate', 'GazeX', 'GazeY', 'Label'])
    df_all.to_csv(os.path.join(output_dir, 'all_data.csv'), index=False)

# Generate and save data
output_dir = 'eye_tracking_data'
num_individuals = 400  # Number of individuals
num_samples_per_person = 4  # Number of samples per individual

generate_and_save_data(num_individuals, num_samples_per_person, output_dir)

# Load the data from CSV
df = pd.read_csv(os.path.join(output_dir, 'all_data.csv'))

# Split the data into features and labels
X = df[['FixationDuration', 'SaccadeAmplitude', 'PupilDilation', 'BlinkRate', 'GazeX', 'GazeY']].values
y = df['Label'].values

# Reshape X for LSTM input [samples, timesteps, features]
X = X.reshape((X.shape[0], 1, X.shape[1]))

# Split the data into 70% train, 15% validation, 15% test
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Build the Bi-LSTM Model with Attention Mechanism
def build_bilstm_attention_model(input_shape):
    inputs = layers.Input(shape=input_shape)

    # Bi-LSTM Layer
    x = layers.Bidirectional(layers.LSTM(128, return_sequences=True))(inputs)

    # Attention Mechanism
    attention = layers.Attention()([x, x])

    # Flatten the output of the attention layer
    x = layers.Flatten()(attention)

    # Dense Layers
    x = layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.001))(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(32, activation='relu', kernel_regularizer=regularizers.l2(0.001))(x)
    x = layers.Dropout(0.5)(x)

    # Output Layer
    outputs = layers.Dense(3, activation='softmax')(x)  # Multi-class classification for 3 classes

    model = models.Model(inputs, outputs)
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    return model

# Build and Train the Model
model = build_bilstm_attention_model((X_train.shape[1], X_train.shape[2]))
history = model.fit(X_train, y_train,
                    epochs=50,
                    batch_size=32,
                    validation_data=(X_val, y_val),
                    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)])

# Evaluate the Model
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {test_acc:.4f}')

# Save model predictions on the test set
predictions = model.predict(X_test)
df_predictions = pd.DataFrame(predictions, columns=['Healthy_Prob', 'Early_Alz_Prob', 'Alz_Prob'])
df_predictions['True_Label'] = y_test
df_predictions.to_csv(os.path.join(output_dir, 'test_predictions.csv'), index=False)

# Plot the Training and Validation Loss
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

# Plot the Training and Validation Accuracy
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# Example evaluation and plotting for 3 different individuals
def plot_individual_data(person_id, df):
    person_df = df[df['PersonID'] == person_id]
    plt.figure(figsize=(10, 6))
    plt.plot(person_df['FixationDuration'], label='Fixation Duration')
    plt.plot(person_df['SaccadeAmplitude'], label='Saccade Amplitude')
    plt.plot(person_df['PupilDilation'], label='Pupil Dilation')
    plt.plot(person_df['BlinkRate'], label='Blink Rate')
    plt.title(f'Person {person_id} Data Over 4 Sessions')
    plt.xlabel('Session')
    plt.ylabel('Metric Value')
    plt.legend()
    plt.show()

# Plot data for an individual with Alzheimer's
plot_individual_data(300, df)

# Plot data for an individual with early-stage Alzheimer's
plot_individual_data(250, df)

# Plot data for a healthy individual
plot_individual_data(100, df)


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models, regularizers
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Set a random seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Data Generation Function
def generate_eye_tracking_data(num_samples, category):
    # Generate synthetic eye-tracking metrics with different distributions based on the category
    if category == 'healthy':
        fixation_duration = np.random.uniform(150, 300, num_samples)  # Healthy range
        saccade_amplitude = np.random.uniform(20, 40, num_samples)    # Healthy range
        pupil_dilation = np.random.uniform(2, 4, num_samples)         # Healthy range
        blink_rate = np.random.uniform(10, 15, num_samples)           # Healthy range
    elif category == 'early_alz':
        fixation_duration = np.random.uniform(300, 450, num_samples)  # Early-stage range
        saccade_amplitude = np.random.uniform(10, 20, num_samples)    # Early-stage range
        pupil_dilation = np.random.uniform(4, 6, num_samples)         # Early-stage range
        blink_rate = np.random.uniform(15, 20, num_samples)           # Early-stage range
    else:  # category == 'alz'
        fixation_duration = np.random.uniform(450, 600, num_samples)  # Alzheimer's range
        saccade_amplitude = np.random.uniform(1, 10, num_samples)     # Alzheimer's range
        pupil_dilation = np.random.uniform(6, 8, num_samples)         # Alzheimer's range
        blink_rate = np.random.uniform(20, 25, num_samples)           # Alzheimer's range

    gaze_x = np.random.uniform(0, 1920, num_samples)              # screen resolution width
    gaze_y = np.random.uniform(0, 1080, num_samples)              # screen resolution height

    # Combine features into a single dataset
    X = np.column_stack((fixation_duration, saccade_amplitude, pupil_dilation, blink_rate, gaze_x, gaze_y))

    # Generate labels (0: Healthy, 1: Early-stage Alzheimer's, 2: Alzheimer's)
    y = 0 if category == 'healthy' else (1 if category == 'early_alz' else 2)

    return X, np.full(num_samples, y)

# Function to generate and save data for multiple individuals
def generate_and_save_data(num_individuals, num_samples_per_person, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    all_data = []

    for i in range(num_individuals):
        if i < 200:
            category = 'healthy'
        elif i < 300:
            category = 'early_alz'
        else:
            category = 'alz'

        X, y = generate_eye_tracking_data(num_samples_per_person, category)
        person_data = np.column_stack((np.full(num_samples_per_person, i), X, y))
        all_data.append(person_data)

        # Save each person's data to a CSV file
        df = pd.DataFrame(person_data, columns=['PersonID', 'FixationDuration', 'SaccadeAmplitude', 'PupilDilation', 'BlinkRate', 'GazeX', 'GazeY', 'Label'])
        df.to_csv(os.path.join(output_dir, f'person_{i}.csv'), index=False)

    # Combine all data into a single dataframe
    all_data = np.vstack(all_data)
    df_all = pd.DataFrame(all_data, columns=['PersonID', 'FixationDuration', 'SaccadeAmplitude', 'PupilDilation', 'BlinkRate', 'GazeX', 'GazeY', 'Label'])
    df_all.to_csv(os.path.join(output_dir, 'all_data.csv'), index=False)

# Generate and save data
output_dir = 'eye_tracking_data'
num_individuals = 400  # Number of individuals
num_samples_per_person = 4  # Number of samples per individual

generate_and_save_data(num_individuals, num_samples_per_person, output_dir)

# Load the data from CSV
df = pd.read_csv(os.path.join(output_dir, 'all_data.csv'))

# Split the data into features and labels
X = df[['FixationDuration', 'SaccadeAmplitude', 'PupilDilation', 'BlinkRate', 'GazeX', 'GazeY']].values
y = df['Label'].values

# Reshape X for LSTM input [samples, timesteps, features]
X = X.reshape((X.shape[0], 1, X.shape[1]))

# Split the data into 70% train, 15% validation, 15% test
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Build the Bi-LSTM Model with Attention Mechanism
def build_bilstm_attention_model(input_shape):
    inputs = layers.Input(shape=input_shape)

    # Bi-LSTM Layer
    x = layers.Bidirectional(layers.LSTM(128, return_sequences=True))(inputs)

    # Attention Mechanism
    attention = layers.Attention()([x, x])

    # Flatten the output of the attention layer
    x = layers.Flatten()(attention)

    # Dense Layers
    x = layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.001))(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(32, activation='relu', kernel_regularizer=regularizers.l2(0.001))(x)
    x = layers.Dropout(0.5)(x)

    # Output Layer
    outputs = layers.Dense(3, activation='softmax')(x)  # Multi-class classification for 3 classes

    model = models.Model(inputs, outputs)
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    return model

# Build and Train the Model
model = build_bilstm_attention_model((X_train.shape[1], X_train.shape[2]))
history = model.fit(X_train, y_train,
                    epochs=50,
                    batch_size=32,
                    validation_data=(X_val, y_val),
                    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)])

# Evaluate the Model
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {test_acc:.4f}')

# Save model predictions on the test set
predictions = model.predict(X_test)
df_predictions = pd.DataFrame(predictions, columns=['Healthy_Prob', 'Early_Alz_Prob', 'Alz_Prob'])
df_predictions['True_Label'] = y_test
df_predictions.to_csv(os.path.join(output_dir, 'test_predictions.csv'), index=False)

# Select three random individuals from the test set for detailed analysis
np.random.seed(42)
random_indices = np.random.choice(df['PersonID'].unique(), 3, replace=False)

# Function to plot and analyze individual data
def analyze_individual(person_id, df, predictions, model):
    person_df = df[df['PersonID'] == person_id]
    true_label = int(person_df['Label'].iloc[0])  # Ensure true_label is an integer

    # Predict the individual's class
    X_person = person_df[['FixationDuration', 'SaccadeAmplitude', 'PupilDilation', 'BlinkRate', 'GazeX', 'GazeY']].values
    X_person = X_person.reshape((X_person.shape[0], 1, X_person.shape[1]))
    person_prediction = model.predict(X_person)
    predicted_label = np.argmax(person_prediction.mean(axis=0))

    # Determine the class
    class_labels = ['Healthy', 'Early Alzheimer\'s', 'Alzheimer\'s']
    true_class = class_labels[true_label]
    predicted_class = class_labels[predicted_label]

    # Generate plots
    sns.set(style="whitegrid")
    plt.figure(figsize=(14, 8))
    plt.plot(person_df['FixationDuration'], label='Fixation Duration', marker='o')
    plt.plot(person_df['SaccadeAmplitude'], label='Saccade Amplitude', marker='s')
    plt.plot(person_df['PupilDilation'], label='Pupil Dilation', marker='D')
    plt.plot(person_df['BlinkRate'], label='Blink Rate', marker='^')
    plt.title(f'Person {person_id} (True: {true_class}, Predicted: {predicted_class})')
    plt.xlabel('Session')
    plt.ylabel('Metric Value')
    plt.legend()
    plt.show()

    # Print analysis report
    print(f"Person {person_id} Analysis Report")
    print(f"True Label: {true_class}")
    print(f"Predicted Label: {predicted_class}")
    print(f"Prediction Probabilities: {person_prediction.mean(axis=0)}")
    print("-----------------------------------------------------------\n")


# Analyze and report on the three selected individuals
for person_id in random_indices:
    analyze_individual(person_id, df, df_predictions, model)


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models, regularizers
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Set a random seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Data Generation Function
def generate_eye_tracking_data(num_samples, category):
    # Generate synthetic eye-tracking metrics with different distributions based on the category
    if category == 'healthy':
        fixation_duration = np.random.uniform(150, 300, num_samples)  # Healthy range
        saccade_amplitude = np.random.uniform(20, 40, num_samples)    # Healthy range
        pupil_dilation = np.random.uniform(2, 4, num_samples)         # Healthy range
        blink_rate = np.random.uniform(10, 15, num_samples)           # Healthy range
    elif category == 'early_alz':
        fixation_duration = np.random.uniform(300, 450, num_samples)  # Early-stage range
        saccade_amplitude = np.random.uniform(10, 20, num_samples)    # Early-stage range
        pupil_dilation = np.random.uniform(4, 6, num_samples)         # Early-stage range
        blink_rate = np.random.uniform(15, 20, num_samples)           # Early-stage range
    else:  # category == 'alz'
        fixation_duration = np.random.uniform(450, 600, num_samples)  # Alzheimer's range
        saccade_amplitude = np.random.uniform(1, 10, num_samples)     # Alzheimer's range
        pupil_dilation = np.random.uniform(6, 8, num_samples)         # Alzheimer's range
        blink_rate = np.random.uniform(20, 25, num_samples)           # Alzheimer's range

    gaze_x = np.random.uniform(0, 1920, num_samples)              # screen resolution width
    gaze_y = np.random.uniform(0, 1080, num_samples)              # screen resolution height

    # Combine features into a single dataset
    X = np.column_stack((fixation_duration, saccade_amplitude, pupil_dilation, blink_rate, gaze_x, gaze_y))

    # Generate labels (0: Healthy, 1: Early-stage Alzheimer's, 2: Alzheimer's)
    y = 0 if category == 'healthy' else (1 if category == 'early_alz' else 2)

    return X, np.full(num_samples, y)

# Function to generate and save data for multiple individuals
def generate_and_save_data(num_individuals, num_samples_per_person, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    all_data = []

    for i in range(num_individuals):
        if i < 200:
            category = 'healthy'
        elif i < 300:
            category = 'early_alz'
        else:
            category = 'alz'

        X, y = generate_eye_tracking_data(num_samples_per_person, category)
        person_data = np.column_stack((np.full(num_samples_per_person, i), X, y))
        all_data.append(person_data)

        # Save each person's data to a CSV file
        df = pd.DataFrame(person_data, columns=['PersonID', 'FixationDuration', 'SaccadeAmplitude', 'PupilDilation', 'BlinkRate', 'GazeX', 'GazeY', 'Label'])
        df.to_csv(os.path.join(output_dir, f'person_{i}.csv'), index=False)

    # Combine all data into a single dataframe
    all_data = np.vstack(all_data)
    df_all = pd.DataFrame(all_data, columns=['PersonID', 'FixationDuration', 'SaccadeAmplitude', 'PupilDilation', 'BlinkRate', 'GazeX', 'GazeY', 'Label'])
    df_all.to_csv(os.path.join(output_dir, 'all_data.csv'), index=False)

# Generate and save data
output_dir = 'eye_tracking_data'
num_individuals = 400  # Number of individuals
num_samples_per_person = 4  # Number of samples per individual

generate_and_save_data(num_individuals, num_samples_per_person, output_dir)

# Load the data from CSV
df = pd.read_csv(os.path.join(output_dir, 'all_data.csv'))

# Split the data into features and labels
X = df[['FixationDuration', 'SaccadeAmplitude', 'PupilDilation', 'BlinkRate', 'GazeX', 'GazeY']].values
y = df['Label'].values

# Reshape X for LSTM input [samples, timesteps, features]
X = X.reshape((X.shape[0], 1, X.shape[1]))

# Split the data into 70% train, 15% validation, 15% test
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Build the Bi-LSTM Model with Attention Mechanism
def build_bilstm_attention_model(input_shape):
    inputs = layers.Input(shape=input_shape)

    # Bi-LSTM Layer
    x = layers.Bidirectional(layers.LSTM(128, return_sequences=True))(inputs)

    # Attention Mechanism
    attention = layers.Attention()([x, x])

    # Flatten the output of the attention layer
    x = layers.Flatten()(attention)

    # Dense Layers
    x = layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.001))(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(32, activation='relu', kernel_regularizer=regularizers.l2(0.001))(x)
    x = layers.Dropout(0.5)(x)

    # Output Layer
    outputs = layers.Dense(3, activation='softmax')(x)  # Multi-class classification for 3 classes

    model = models.Model(inputs, outputs)
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    return model

# Build and Train the Model
model = build_bilstm_attention_model((X_train.shape[1], X_train.shape[2]))
history = model.fit(X_train, y_train,
                    epochs=50,
                    batch_size=32,
                    validation_data=(X_val, y_val),
                    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)])

# Evaluate the Model
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {test_acc:.4f}')

# Save model predictions on the test set
predictions = model.predict(X_test)
df_predictions = pd.DataFrame(predictions, columns=['Healthy_Prob', 'Early_Alz_Prob', 'Alz_Prob'])
df_predictions['True_Label'] = y_test
df_predictions.to_csv(os.path.join(output_dir, 'test_predictions.csv'), index=False)

# Generate data for three new individuals (not part of the original 400)
new_individuals_data = []
categories = ['healthy', 'early_alz', 'alz']

for i, category in enumerate(categories):
    X_new, y_new = generate_eye_tracking_data(4, category)  # 4 samples per person
    new_individuals_data.append((X_new, y_new, category))

# Function to plot and analyze individual data
def analyze_new_individual(X_new, y_new, category, model):
    # Predict the individual's class
    X_person = X_new.reshape((X_new.shape[0], 1, X_new.shape[1]))
    person_prediction = model.predict(X_person)
    predicted_label = np.argmax(person_prediction.mean(axis=0))

    # Determine the class
    class_labels = ['Healthy', 'Early Alzheimer\'s', 'Alzheimer\'s']
    true_class = class_labels[y_new[0]]
    predicted_class = class_labels[predicted_label]

    # Generate plots
    sns.set(style="whitegrid")
    plt.figure(figsize=(14, 8))
    plt.plot(X_new[:, 0], label='Fixation Duration', marker='o')
    plt.plot(X_new[:, 1], label='Saccade Amplitude', marker='s')
    plt.plot(X_new[:, 2], label='Pupil Dilation', marker='D')
    plt.plot(X_new[:, 3], label='Blink Rate', marker='^')
    plt.title(f'New Individual ({category.capitalize()}) - Predicted: {predicted_class}')
    plt.xlabel('Session')
    plt.ylabel('Metric Value')
    plt.legend()
    plt.show()

    # Print analysis report
    print(f"New Individual ({category.capitalize()}) Analysis Report")
    print(f"True Label: {true_class}")
    print(f"Predicted Label: {predicted_class}")
    print(f"Prediction Probabilities: {person_prediction.mean(axis=0)}")
    print("-----------------------------------------------------------\n")

# Analyze and report on the three new individuals
for X_new, y_new, category in new_individuals_data:
    analyze_new_individual(X_new, y_new, category, model)


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models, regularizers
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Set a random seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Data Generation Function
def generate_eye_tracking_data(num_samples, category):
    # Generate synthetic eye-tracking metrics with different distributions based on the category
    if category == 'healthy':
        fixation_duration = np.random.uniform(150, 300, num_samples)  # Healthy range
        saccade_amplitude = np.random.uniform(20, 40, num_samples)    # Healthy range
        pupil_dilation = np.random.uniform(2, 4, num_samples)         # Healthy range
        blink_rate = np.random.uniform(10, 15, num_samples)           # Healthy range
    elif category == 'early_alz':
        fixation_duration = np.random.uniform(300, 450, num_samples)  # Early-stage range
        saccade_amplitude = np.random.uniform(10, 20, num_samples)    # Early-stage range
        pupil_dilation = np.random.uniform(4, 6, num_samples)         # Early-stage range
        blink_rate = np.random.uniform(15, 20, num_samples)           # Early-stage range
    else:  # category == 'alz'
        fixation_duration = np.random.uniform(450, 600, num_samples)  # Alzheimer's range
        saccade_amplitude = np.random.uniform(1, 10, num_samples)     # Alzheimer's range
        pupil_dilation = np.random.uniform(6, 8, num_samples)         # Alzheimer's range
        blink_rate = np.random.uniform(20, 25, num_samples)           # Alzheimer's range

    gaze_x = np.random.uniform(0, 1920, num_samples)              # screen resolution width
    gaze_y = np.random.uniform(0, 1080, num_samples)              # screen resolution height

    # Combine features into a single dataset
    X = np.column_stack((fixation_duration, saccade_amplitude, pupil_dilation, blink_rate, gaze_x, gaze_y))

    # Generate labels (0: Healthy, 1: Early-stage Alzheimer's, 2: Alzheimer's)
    y = 0 if category == 'healthy' else (1 if category == 'early_alz' else 2)

    return X, np.full(num_samples, y)

# Function to generate and save data for multiple individuals
def generate_and_save_data(num_individuals, num_samples_per_person, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    all_data = []

    for i in range(num_individuals):
        if i < 200:
            category = 'healthy'
        elif i < 300:
            category = 'early_alz'
        else:
            category = 'alz'

        X, y = generate_eye_tracking_data(num_samples_per_person, category)
        person_data = np.column_stack((np.full(num_samples_per_person, i), X, y))
        all_data.append(person_data)

        # Save each person's data to a CSV file
        df = pd.DataFrame(person_data, columns=['PersonID', 'FixationDuration', 'SaccadeAmplitude', 'PupilDilation', 'BlinkRate', 'GazeX', 'GazeY', 'Label'])
        df.to_csv(os.path.join(output_dir, f'person_{i}.csv'), index=False)

    # Combine all data into a single dataframe
    all_data = np.vstack(all_data)
    df_all = pd.DataFrame(all_data, columns=['PersonID', 'FixationDuration', 'SaccadeAmplitude', 'PupilDilation', 'BlinkRate', 'GazeX', 'GazeY', 'Label'])
    df_all.to_csv(os.path.join(output_dir, 'all_data.csv'), index=False)

# Generate and save data
output_dir = 'eye_tracking_data'
num_individuals = 400  # Number of individuals
num_samples_per_person = 4  # Number of samples per individual

generate_and_save_data(num_individuals, num_samples_per_person, output_dir)

# Load the data from CSV
df = pd.read_csv(os.path.join(output_dir, 'all_data.csv'))

# Split the data into features and labels
X = df[['FixationDuration', 'SaccadeAmplitude', 'PupilDilation', 'BlinkRate', 'GazeX', 'GazeY']].values
y = df['Label'].values

# Reshape X for LSTM input [samples, timesteps, features]
X = X.reshape((X.shape[0], 1, X.shape[1]))

# Split the data into 70% train, 15% validation, 15% test
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Build the Bi-LSTM Model with Attention Mechanism
def build_bilstm_attention_model(input_shape):
    inputs = layers.Input(shape=input_shape)

    # Bi-LSTM Layer
    x = layers.Bidirectional(layers.LSTM(128, return_sequences=True))(inputs)

    # Attention Mechanism
    attention = layers.Attention()([x, x])

    # Flatten the output of the attention layer
    x = layers.Flatten()(attention)

    # Dense Layers
    x = layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.001))(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(32, activation='relu', kernel_regularizer=regularizers.l2(0.001))(x)
    x = layers.Dropout(0.5)(x)

    # Output Layer
    outputs = layers.Dense(3, activation='softmax')(x)  # Multi-class classification for 3 classes

    model = models.Model(inputs, outputs)
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    return model

# Build and Train the Model
model = build_bilstm_attention_model((X_train.shape[1], X_train.shape[2]))
history = model.fit(X_train, y_train,
                    epochs=50,
                    batch_size=32,
                    validation_data=(X_val, y_val),
                    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)])

# Evaluate the Model
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {test_acc:.4f}')

# Save model predictions on the test set
predictions = model.predict(X_test)
df_predictions = pd.DataFrame(predictions, columns=['Healthy_Prob', 'Early_Alz_Prob', 'Alz_Prob'])
df_predictions['True_Label'] = y_test
df_predictions.to_csv(os.path.join(output_dir, 'test_predictions.csv'), index=False)

# Generate data for three new individuals (not part of the original 400)
new_individuals_data = []
categories = ['healthy', 'early_alz', 'alz']

for i, category in enumerate(categories):
    X_new, y_new = generate_eye_tracking_data(4, category)  # 4 samples per person
    new_individuals_data.append((X_new, y_new, category))

# Function to plot and analyze individual data
def analyze_new_individual(X_new, y_new, category, model):
    # Predict the individual's class
    X_person = X_new.reshape((X_new.shape[0], 1, X_new.shape[1]))
    person_prediction = model.predict(X_person)
    predicted_label = np.argmax(person_prediction.mean(axis=0))

    # Determine the class
    class_labels = ['Healthy', 'Early Alzheimer\'s', 'Alzheimer\'s']
    true_class = class_labels[y_new[0]]
    predicted_class = class_labels[predicted_label]

    # Generate plots
    sns.set(style="whitegrid")
    plt.figure(figsize=(14, 8))
    plt.plot(X_new[:, 0], label='Fixation Duration', marker='o')
    plt.plot(X_new[:, 1], label='Saccade Amplitude', marker='s')
    plt.plot(X_new[:, 2], label='Pupil Dilation', marker='D')
    plt.plot(X_new[:, 3], label='Blink Rate', marker='^')
    plt.title(f'New Individual ({category.capitalize()}) - Predicted: {predicted_class}')
    plt.xlabel('Session')
    plt.ylabel('Metric Value')
    plt.legend()
    plt.show()

    # Print analysis report
    print(f"New Individual ({category.capitalize()}) Analysis Report")
    print(f"True Label: {true_class}")
    print(f"Predicted Label: {predicted_class}")
    print(f"Prediction Probabilities: {person_prediction.mean(axis=0)}")
    print("-----------------------------------------------------------")

    # Interpretation based on the plot and metrics
    print("Interpretation:")
    if predicted_class == 'Healthy':
        print("The eye-tracking metrics for this individual are within normal ranges, indicating no signs of Alzheimer's.")
    elif predicted_class == 'Early Alzheimer\'s':
        print("The eye-tracking metrics suggest some early-stage cognitive decline, which could be indicative of early Alzheimer's.")
    else:  # Alzheimer's
        print("The eye-tracking metrics show significant deviations from normal, consistent with Alzheimer's disease.")
    print("-----------------------------------------------------------\n")

# Analyze and report on the three new individuals
for X_new, y_new, category in new_individuals_data:
    analyze_new_individual(X_new, y_new, category, model)


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models, regularizers
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Set a random seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Data Generation Function
def generate_eye_tracking_data(num_samples, category):
    # Generate synthetic eye-tracking metrics with different distributions based on the category
    if category == 'healthy':
        fixation_duration = np.random.uniform(150, 300, num_samples)  # Healthy range
        saccade_amplitude = np.random.uniform(20, 40, num_samples)    # Healthy range
        pupil_dilation = np.random.uniform(2, 4, num_samples)         # Healthy range
        blink_rate = np.random.uniform(10, 15, num_samples)           # Healthy range
    elif category == 'early_alz':
        fixation_duration = np.random.uniform(300, 450, num_samples)  # Early-stage range
        saccade_amplitude = np.random.uniform(10, 20, num_samples)    # Early-stage range
        pupil_dilation = np.random.uniform(4, 6, num_samples)         # Early-stage range
        blink_rate = np.random.uniform(15, 20, num_samples)           # Early-stage range
    else:  # category == 'alz'
        fixation_duration = np.random.uniform(450, 600, num_samples)  # Alzheimer's range
        saccade_amplitude = np.random.uniform(1, 10, num_samples)     # Alzheimer's range
        pupil_dilation = np.random.uniform(6, 8, num_samples)         # Alzheimer's range
        blink_rate = np.random.uniform(20, 25, num_samples)           # Alzheimer's range

    gaze_x = np.random.uniform(0, 1920, num_samples)              # screen resolution width
    gaze_y = np.random.uniform(0, 1080, num_samples)              # screen resolution height

    # Combine features into a single dataset
    X = np.column_stack((fixation_duration, saccade_amplitude, pupil_dilation, blink_rate, gaze_x, gaze_y))

    # Generate labels (0: Healthy, 1: Early-stage Alzheimer's, 2: Alzheimer's)
    y = 0 if category == 'healthy' else (1 if category == 'early_alz' else 2)

    return X, np.full(num_samples, y)

# Function to generate and save data for multiple individuals
def generate_and_save_data(num_individuals, num_samples_per_person, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    all_data = []

    for i in range(num_individuals):
        if i < 200:
            category = 'healthy'
        elif i < 300:
            category = 'early_alz'
        else:
            category = 'alz'

        X, y = generate_eye_tracking_data(num_samples_per_person, category)
        person_data = np.column_stack((np.full(num_samples_per_person, i), X, y))
        all_data.append(person_data)

        # Save each person's data to a CSV file
        df = pd.DataFrame(person_data, columns=['PersonID', 'FixationDuration', 'SaccadeAmplitude', 'PupilDilation', 'BlinkRate', 'GazeX', 'GazeY', 'Label'])
        df.to_csv(os.path.join(output_dir, f'person_{i}.csv'), index=False)

    # Combine all data into a single dataframe
    all_data = np.vstack(all_data)
    df_all = pd.DataFrame(all_data, columns=['PersonID', 'FixationDuration', 'SaccadeAmplitude', 'PupilDilation', 'BlinkRate', 'GazeX', 'GazeY', 'Label'])
    df_all.to_csv(os.path.join(output_dir, 'all_data.csv'), index=False)

# Generate and save data
output_dir = 'eye_tracking_data'
num_individuals = 400  # Number of individuals
num_samples_per_person = 4  # Number of samples per individual

generate_and_save_data(num_individuals, num_samples_per_person, output_dir)

# Load the data from CSV
df = pd.read_csv(os.path.join(output_dir, 'all_data.csv'))

# Split the data into features and labels
X = df[['FixationDuration', 'SaccadeAmplitude', 'PupilDilation', 'BlinkRate', 'GazeX', 'GazeY']].values
y = df['Label'].values

# Reshape X for LSTM input [samples, timesteps, features]
X = X.reshape((X.shape[0], 1, X.shape[1]))

# Split the data into 70% train, 15% validation, 15% test
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Build the Bi-LSTM Model with Attention Mechanism
def build_bilstm_attention_model(input_shape):
    inputs = layers.Input(shape=input_shape)

    # Bi-LSTM Layer
    x = layers.Bidirectional(layers.LSTM(128, return_sequences=True))(inputs)

    # Attention Mechanism
    attention = layers.Attention()([x, x])

    # Flatten the output of the attention layer
    x = layers.Flatten()(attention)

    # Dense Layers
    x = layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.001))(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(32, activation='relu', kernel_regularizer=regularizers.l2(0.001))(x)
    x = layers.Dropout(0.5)(x)

    # Output Layer
    outputs = layers.Dense(3, activation='softmax')(x)  # Multi-class classification for 3 classes

    model = models.Model(inputs, outputs)
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    return model

# Build and Train the Model
model = build_bilstm_attention_model((X_train.shape[1], X_train.shape[2]))
history = model.fit(X_train, y_train,
                    epochs=50,
                    batch_size=32,
                    validation_data=(X_val, y_val),
                    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)])

# Evaluate the Model
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {test_acc:.4f}')

# Save model predictions on the test set
predictions = model.predict(X_test)
df_predictions = pd.DataFrame(predictions, columns=['Healthy_Prob', 'Early_Alz_Prob', 'Alz_Prob'])
df_predictions['True_Label'] = y_test
df_predictions.to_csv(os.path.join(output_dir, 'test_predictions.csv'), index=False)

# Generate data for three new individuals (not part of the original 400)
new_individuals_data = []
categories = ['healthy', 'early_alz', 'alz']

for i, category in enumerate(categories):
    X_new, y_new = generate_eye_tracking_data(4, category)  # 4 samples per person
    new_individuals_data.append((X_new, y_new, category))

# Function to plot and analyze individual data
def analyze_new_individual(X_new, y_new, category, model):
    # Predict the individual's class
    X_person = X_new.reshape((X_new.shape[0], 1, X_new.shape[1]))
    person_prediction = model.predict(X_person)
    predicted_label = np.argmax(person_prediction.mean(axis=0))

    # Determine the class
    class_labels = ['Healthy', 'Early Alzheimer\'s', 'Alzheimer\'s']
    true_class = class_labels[y_new[0]]
    predicted_class = class_labels[predicted_label]

    # Generate line plots for each metric
    sns.set(style="whitegrid")
    plt.figure(figsize=(14, 8))
    plt.plot(X_new[:, 0], label='Fixation Duration', marker='o')
    plt.plot(X_new[:, 1], label='Saccade Amplitude', marker='s')
    plt.plot(X_new[:, 2], label='Pupil Dilation', marker='D')
    plt.plot(X_new[:, 3], label='Blink Rate', marker='^')
    plt.title(f'New Individual ({category.capitalize()}) - Predicted: {predicted_class}')
    plt.xlabel('Session')
    plt.ylabel('Metric Value')
    plt.legend()
    plt.show()

    # Generate heatmap of the correlations between metrics
    corr_matrix = pd.DataFrame(X_new, columns=['Fixation Duration', 'Saccade Amplitude', 'Pupil Dilation', 'Blink Rate', 'Gaze X', 'Gaze Y']).corr()
    plt.figure(figsize=(10, 8))
    sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', cbar=True)
    plt.title(f'Correlation Heatmap for {category.capitalize()} Individual')
    plt.show()

    # Generate distribution plot for each metric
    plt.figure(figsize=(14, 8))
    for i, column in enumerate(['Fixation Duration', 'Saccade Amplitude', 'Pupil Dilation', 'Blink Rate']):
        sns.kdeplot(X_new[:, i], label=column, shade=True)
    plt.title(f'Distribution of Metrics for {category.capitalize()} Individual')
    plt.xlabel('Metric Value')
    plt.ylabel('Density')
    plt.legend()
    plt.show()

    # Print analysis report
    print(f"New Individual ({category.capitalize()}) Analysis Report")
    print(f"True Label: {true_class}")
    print(f"Predicted Label: {predicted_class}")
    print(f"Prediction Probabilities: {person_prediction.mean(axis=0)}")
    print("-----------------------------------------------------------")

    # Interpretation based on the plot and metrics
    print("Interpretation:")
    if predicted_class == 'Healthy':
        print("The eye-tracking metrics for this individual are within normal ranges, indicating no signs of Alzheimer's.")
    elif predicted_class == 'Early Alzheimer\'s':
        print("The eye-tracking metrics suggest some early-stage cognitive decline, which could be indicative of early Alzheimer's.")
    else:  # Alzheimer's
        print("The eye-tracking metrics show significant deviations from normal, consistent with Alzheimer's disease.")
    print("-----------------------------------------------------------\n")

# Analyze and report on the three new individuals
for X_new, y_new, category in new_individuals_data:
    analyze_new_individual(X_new, y_new, category, model)


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models, regularizers
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Set a random seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Data Generation Function
def generate_eye_tracking_data(num_samples, category):
    # Generate synthetic eye-tracking metrics with different distributions based on the category
    if category == 'healthy':
        fixation_duration = np.random.uniform(150, 300, num_samples)  # Healthy range
        saccade_amplitude = np.random.uniform(20, 40, num_samples)    # Healthy range
        pupil_dilation = np.random.uniform(2, 4, num_samples)         # Healthy range
        blink_rate = np.random.uniform(10, 15, num_samples)           # Healthy range
    elif category == 'early_alz':
        fixation_duration = np.random.uniform(300, 450, num_samples)  # Early-stage range
        saccade_amplitude = np.random.uniform(10, 20, num_samples)    # Early-stage range
        pupil_dilation = np.random.uniform(4, 6, num_samples)         # Early-stage range
        blink_rate = np.random.uniform(15, 20, num_samples)           # Early-stage range
    else:  # category == 'alz'
        fixation_duration = np.random.uniform(450, 600, num_samples)  # Alzheimer's range
        saccade_amplitude = np.random.uniform(1, 10, num_samples)     # Alzheimer's range
        pupil_dilation = np.random.uniform(6, 8, num_samples)         # Alzheimer's range
        blink_rate = np.random.uniform(20, 25, num_samples)           # Alzheimer's range

    gaze_x = np.random.uniform(0, 1920, num_samples)              # screen resolution width
    gaze_y = np.random.uniform(0, 1080, num_samples)              # screen resolution height

    # Combine features into a single dataset
    X = np.column_stack((fixation_duration, saccade_amplitude, pupil_dilation, blink_rate, gaze_x, gaze_y))

    # Generate labels (0: Healthy, 1: Early-stage Alzheimer's, 2: Alzheimer's)
    y = 0 if category == 'healthy' else (1 if category == 'early_alz' else 2)

    return X, np.full(num_samples, y)

# Function to generate and save data for multiple individuals
def generate_and_save_data(num_individuals, num_samples_per_person, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    all_data = []

    for i in range(num_individuals):
        if i < 200:
            category = 'healthy'
        elif i < 300:
            category = 'early_alz'
        else:
            category = 'alz'

        X, y = generate_eye_tracking_data(num_samples_per_person, category)
        person_data = np.column_stack((np.full(num_samples_per_person, i), X, y))
        all_data.append(person_data)

        # Save each person's data to a CSV file
        df = pd.DataFrame(person_data, columns=['PersonID', 'FixationDuration', 'SaccadeAmplitude', 'PupilDilation', 'BlinkRate', 'GazeX', 'GazeY', 'Label'])
        df.to_csv(os.path.join(output_dir, f'person_{i}.csv'), index=False)

    # Combine all data into a single dataframe
    all_data = np.vstack(all_data)
    df_all = pd.DataFrame(all_data, columns=['PersonID', 'FixationDuration', 'SaccadeAmplitude', 'PupilDilation', 'BlinkRate', 'GazeX', 'GazeY', 'Label'])
    df_all.to_csv(os.path.join(output_dir, 'all_data.csv'), index=False)

# Generate and save data
output_dir = 'eye_tracking_data'
num_individuals = 400  # Number of individuals
num_samples_per_person = 4  # Number of samples per individual

generate_and_save_data(num_individuals, num_samples_per_person, output_dir)

# Load the data from CSV
df = pd.read_csv(os.path.join(output_dir, 'all_data.csv'))

# Split the data into features and labels
X = df[['FixationDuration', 'SaccadeAmplitude', 'PupilDilation', 'BlinkRate', 'GazeX', 'GazeY']].values
y = df['Label'].values

# Reshape X for LSTM input [samples, timesteps, features]
X = X.reshape((X.shape[0], 1, X.shape[1]))

# Split the data into 70% train, 15% validation, 15% test
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Build the Bi-LSTM Model with Attention Mechanism
def build_bilstm_attention_model(input_shape):
    inputs = layers.Input(shape=input_shape)

    # Bi-LSTM Layer
    x = layers.Bidirectional(layers.LSTM(128, return_sequences=True))(inputs)

    # Attention Mechanism
    attention = layers.Attention()([x, x])

    # Flatten the output of the attention layer
    x = layers.Flatten()(attention)

    # Dense Layers
    x = layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.001))(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(32, activation='relu', kernel_regularizer=regularizers.l2(0.001))(x)
    x = layers.Dropout(0.5)(x)

    # Output Layer
    outputs = layers.Dense(3, activation='softmax')(x)  # Multi-class classification for 3 classes

    model = models.Model(inputs, outputs)
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    return model

# Build and Train the Model
model = build_bilstm_attention_model((X_train.shape[1], X_train.shape[2]))
history = model.fit(X_train, y_train,
                    epochs=50,
                    batch_size=32,
                    validation_data=(X_val, y_val),
                    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)])

# Evaluate the Model
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {test_acc:.4f}')

# Save model predictions on the test set
predictions = model.predict(X_test)
df_predictions = pd.DataFrame(predictions, columns=['Healthy_Prob', 'Early_Alz_Prob', 'Alz_Prob'])
df_predictions['True_Label'] = y_test
df_predictions.to_csv(os.path.join(output_dir, 'test_predictions.csv'), index=False)

# Generate data for three new individuals (not part of the original 400)
new_individuals_data = []
categories = ['healthy', 'early_alz', 'alz']

for i, category in enumerate(categories):
    X_new, y_new = generate_eye_tracking_data(4, category)  # 4 samples per person
    new_individuals_data.append((X_new, y_new, category))

# Function to plot and analyze individual data
def analyze_new_individual(X_new, y_new, category, model):
    # Predict the individual's class
    X_person = X_new.reshape((X_new.shape[0], 1, X_new.shape[1]))
    person_prediction = model.predict(X_person)
    predicted_label = np.argmax(person_prediction.mean(axis=0))

    # Determine the class
    class_labels = ['Healthy', 'Early Alzheimer\'s', 'Alzheimer\'s']
    true_class = class_labels[y_new[0]]
    predicted_class = class_labels[predicted_label]

    # Generate line plots for each metric
    sns.set(style="whitegrid")
    plt.figure(figsize=(14, 8))
    plt.plot(X_new[:, 0], label='Fixation Duration', marker='o', color='b', linestyle='-', linewidth=2)
    plt.plot(X_new[:, 1], label='Saccade Amplitude', marker='s', color='g', linestyle='--', linewidth=2)
    plt.plot(X_new[:, 2], label='Pupil Dilation', marker='D', color='r', linestyle='-.', linewidth=2)
    plt.plot(X_new[:, 3], label='Blink Rate', marker='^', color='m', linestyle=':', linewidth=2)
    plt.title(f'New Individual ({category.capitalize()}) - Predicted: {predicted_class}', fontsize=16)
    plt.xlabel('Session', fontsize=14)
    plt.ylabel('Metric Value', fontsize=14)
    plt.legend(fontsize=12)
    plt.grid(True)
    plt.show()

    # Generate heatmap of the correlations between metrics
    corr_matrix = pd.DataFrame(X_new, columns=['Fixation Duration', 'Saccade Amplitude', 'Pupil Dilation', 'Blink Rate', 'Gaze X', 'Gaze Y']).corr()
    plt.figure(figsize=(10, 8))
    sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', cbar=True, annot_kws={"size": 12})
    plt.title(f'Correlation Heatmap for {category.capitalize()} Individual', fontsize=16)
    plt.show()

    # Generate distribution plot for each metric
    plt.figure(figsize=(14, 8))
    for i, column in enumerate(['Fixation Duration', 'Saccade Amplitude', 'Pupil Dilation', 'Blink Rate']):
        sns.kdeplot(X_new[:, i], label=column, shade=True, linewidth=2)
    plt.title(f'Distribution of Metrics for {category.capitalize()} Individual', fontsize=16)
    plt.xlabel('Metric Value', fontsize=14)
    plt.ylabel('Density', fontsize=14)
    plt.legend(fontsize=12)
    plt.grid(True)
    plt.show()

    # Print analysis report
    print(f"New Individual ({category.capitalize()}) Analysis Report")
    print(f"True Label: {true_class}")
    print(f"Predicted Label: {predicted_class}")
    print(f"Prediction Probabilities: {person_prediction.mean(axis=0)}")
    print("-----------------------------------------------------------")

    # Interpretation based on the plot and metrics
    print("Interpretation:")
    if predicted_class == 'Healthy':
        print("The eye-tracking metrics for this individual are within normal ranges, indicating no signs of Alzheimer's. The person likely does not have Alzheimer's disease based on the observed data.")
    elif predicted_class == 'Early Alzheimer\'s':
        print("The eye-tracking metrics suggest some early-stage cognitive decline. These patterns are consistent with early Alzheimer's disease, indicating that the person might be in the early stages of the condition.")
    else:  # Alzheimer's
        print("The eye-tracking metrics show significant deviations from normal ranges, with patterns consistent with Alzheimer's disease. The person likely has Alzheimer's disease based on the observed data.")
    print("-----------------------------------------------------------\n")

# Analyze and report on the three new individuals
for X_new, y_new, category in new_individuals_data:
    analyze_new_individual(X_new, y_new, category, model)


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models, regularizers
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Set a random seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Data Generation Function
def generate_eye_tracking_data(num_samples, category):
    # Generate synthetic eye-tracking metrics with different distributions based on the category
    if category == 'healthy':
        fixation_duration = np.random.uniform(150, 300, num_samples)  # Healthy range
        saccade_amplitude = np.random.uniform(20, 40, num_samples)    # Healthy range
        pupil_dilation = np.random.uniform(2, 4, num_samples)         # Healthy range
        blink_rate = np.random.uniform(10, 15, num_samples)           # Healthy range
    elif category == 'early_alz':
        fixation_duration = np.random.uniform(300, 450, num_samples)  # Early-stage range
        saccade_amplitude = np.random.uniform(10, 20, num_samples)    # Early-stage range
        pupil_dilation = np.random.uniform(4, 6, num_samples)         # Early-stage range
        blink_rate = np.random.uniform(15, 20, num_samples)           # Early-stage range
    else:  # category == 'alz'
        fixation_duration = np.random.uniform(450, 600, num_samples)  # Alzheimer's range
        saccade_amplitude = np.random.uniform(1, 10, num_samples)     # Alzheimer's range
        pupil_dilation = np.random.uniform(6, 8, num_samples)         # Alzheimer's range
        blink_rate = np.random.uniform(20, 25, num_samples)           # Alzheimer's range

    gaze_x = np.random.uniform(0, 1920, num_samples)              # screen resolution width
    gaze_y = np.random.uniform(0, 1080, num_samples)              # screen resolution height

    # Combine features into a single dataset
    X = np.column_stack((fixation_duration, saccade_amplitude, pupil_dilation, blink_rate, gaze_x, gaze_y))

    # Generate labels (0: Healthy, 1: Early-stage Alzheimer's, 2: Alzheimer's)
    y = 0 if category == 'healthy' else (1 if category == 'early_alz' else 2)

    return X, np.full(num_samples, y)

# Function to generate and save data for multiple individuals
def generate_and_save_data(num_individuals, num_samples_per_person, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    all_data = []

    for i in range(num_individuals):
        if i < 200:
            category = 'healthy'
        elif i < 300:
            category = 'early_alz'
        else:
            category = 'alz'

        X, y = generate_eye_tracking_data(num_samples_per_person, category)
        person_data = np.column_stack((np.full(num_samples_per_person, i), X, y))
        all_data.append(person_data)

        # Save each person's data to a CSV file
        df = pd.DataFrame(person_data, columns=['PersonID', 'FixationDuration', 'SaccadeAmplitude', 'PupilDilation', 'BlinkRate', 'GazeX', 'GazeY', 'Label'])
        df.to_csv(os.path.join(output_dir, f'person_{i}.csv'), index=False)

    # Combine all data into a single dataframe
    all_data = np.vstack(all_data)
    df_all = pd.DataFrame(all_data, columns=['PersonID', 'FixationDuration', 'SaccadeAmplitude', 'PupilDilation', 'BlinkRate', 'GazeX', 'GazeY', 'Label'])
    df_all.to_csv(os.path.join(output_dir, 'all_data.csv'), index=False)

# Generate and save data
output_dir = 'eye_tracking_data'
num_individuals = 400  # Number of individuals
num_samples_per_person = 4  # Number of samples per individual

generate_and_save_data(num_individuals, num_samples_per_person, output_dir)

# Load the data from CSV
df = pd.read_csv(os.path.join(output_dir, 'all_data.csv'))

# Split the data into features and labels
X = df[['FixationDuration', 'SaccadeAmplitude', 'PupilDilation', 'BlinkRate', 'GazeX', 'GazeY']].values
y = df['Label'].values

# Reshape X for LSTM input [samples, timesteps, features]
X = X.reshape((X.shape[0], 1, X.shape[1]))

# Split the data into 70% train, 15% validation, 15% test
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Build the Bi-LSTM Model with Attention Mechanism
def build_bilstm_attention_model(input_shape):
    inputs = layers.Input(shape=input_shape)

    # Bi-LSTM Layer
    x = layers.Bidirectional(layers.LSTM(128, return_sequences=True))(inputs)

    # Attention Mechanism
    attention = layers.Attention()([x, x])

    # Flatten the output of the attention layer
    x = layers.Flatten()(attention)

    # Dense Layers
    x = layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.001))(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(32, activation='relu', kernel_regularizer=regularizers.l2(0.001))(x)
    x = layers.Dropout(0.5)(x)

    # Output Layer
    outputs = layers.Dense(3, activation='softmax')(x)  # Multi-class classification for 3 classes

    model = models.Model(inputs, outputs)
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    return model

# Build and Train the Model
model = build_bilstm_attention_model((X_train.shape[1], X_train.shape[2]))
history = model.fit(X_train, y_train,
                    epochs=50,
                    batch_size=32,
                    validation_data=(X_val, y_val),
                    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)])

# Evaluate the Model
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {test_acc:.4f}')

# Save model predictions on the test set
predictions = model.predict(X_test)
df_predictions = pd.DataFrame(predictions, columns=['Healthy_Prob', 'Early_Alz_Prob', 'Alz_Prob'])
df_predictions['True_Label'] = y_test
df_predictions.to_csv(os.path.join(output_dir, 'test_predictions.csv'), index=False)

# Generate data for three new individuals (not part of the original 400)
new_individuals_data = []
categories = ['healthy', 'early_alz', 'alz']

for i, category in enumerate(categories):
    X_new, y_new = generate_eye_tracking_data(4, category)  # 4 samples per person
    new_individuals_data.append((X_new, y_new, category))

# Function to plot and analyze individual data
def analyze_new_individual(X_new, y_new, category, model):
    # Predict the individual's class
    X_person = X_new.reshape((X_new.shape[0], 1, X_new.shape[1]))
    person_prediction = model.predict(X_person)
    predicted_label = np.argmax(person_prediction.mean(axis=0))

    # Determine the class
    class_labels = ['Healthy', 'Early Alzheimer\'s', 'Alzheimer\'s']
    true_class = class_labels[y_new[0]]
    predicted_class = class_labels[predicted_label]

    # Generate line plots for each metric
    sns.set(style="whitegrid")
    plt.figure(figsize=(14, 8))
    plt.plot(X_new[:, 0], label='Fixation Duration', marker='o', color='b', linestyle='-', linewidth=2)
    plt.plot(X_new[:, 1], label='Saccade Amplitude', marker='s', color='g', linestyle='--', linewidth=2)
    plt.plot(X_new[:, 2], label='Pupil Dilation', marker='D', color='r', linestyle='-.', linewidth=2)
    plt.plot(X_new[:, 3], label='Blink Rate', marker='^', color='m', linestyle=':', linewidth=2)
    plt.title(f'New Individual ({category.capitalize()}) - Predicted: {predicted_class}', fontsize=16)
    plt.xlabel('Session', fontsize=14)
    plt.ylabel('Metric Value', fontsize=14)
    plt.legend(fontsize=12)
    plt.grid(True)
    plt.show()

    # Generate heatmap of the correlations between metrics
    corr_matrix = pd.DataFrame(X_new, columns=['Fixation Duration', 'Saccade Amplitude', 'Pupil Dilation', 'Blink Rate', 'Gaze X', 'Gaze Y']).corr()
    plt.figure(figsize=(10, 8))
    sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', cbar=True, annot_kws={"size": 12})
    plt.title(f'Correlation Heatmap for {category.capitalize()} Individual', fontsize=16)
    plt.show()

    # Generate distribution plot for each metric
    plt.figure(figsize=(14, 8))
    for i, column in enumerate(['Fixation Duration', 'Saccade Amplitude', 'Pupil Dilation', 'Blink Rate']):
        sns.kdeplot(X_new[:, i], label=column, fill=True, linewidth=2)
    plt.title(f'Distribution of Metrics for {category.capitalize()} Individual', fontsize=16)
    plt.xlabel('Metric Value', fontsize=14)
    plt.ylabel('Density', fontsize=14)
    plt.legend(fontsize=12)
    plt.grid(True)
    plt.show()

    # Print analysis report
    print(f"New Individual ({category.capitalize()}) Analysis Report")
    print(f"True Label: {true_class}")
    print(f"Predicted Label: {predicted_class}")
    print(f"Prediction Probabilities: {person_prediction.mean(axis=0)}")
    print("-----------------------------------------------------------")

    # Interpretation based on the plot and metrics
    print("Interpretation:")
    if predicted_class == 'Healthy':
        print("The eye-tracking metrics for this individual are within normal ranges, indicating no signs of Alzheimer's. The person likely does not have Alzheimer's disease based on the observed data.")
    elif predicted_class == 'Early Alzheimer\'s':
        print("The eye-tracking metrics suggest some early-stage cognitive decline. These patterns are consistent with early Alzheimer's disease, indicating that the person might be in the early stages of the condition.")
    else:  # Alzheimer's
        print("The eye-tracking metrics show significant deviations from normal ranges, with patterns consistent with Alzheimer's disease. The person likely has Alzheimer's disease based on the observed data.")
    print("-----------------------------------------------------------\n")

# Analyze and report on the three new individuals
for X_new, y_new, category in new_individuals_data:
    analyze_new_individual(X_new, y_new, category, model)


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models, regularizers
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
from math import pi
import os

# Set a random seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Data Generation Function
def generate_eye_tracking_data(num_samples, category):
    # Generate synthetic eye-tracking metrics with different distributions based on the category
    if category == 'healthy':
        fixation_duration = np.random.uniform(150, 300, num_samples)  # Healthy range
        saccade_amplitude = np.random.uniform(20, 40, num_samples)    # Healthy range
        pupil_dilation = np.random.uniform(2, 4, num_samples)         # Healthy range
        blink_rate = np.random.uniform(10, 15, num_samples)           # Healthy range
    elif category == 'early_alz':
        fixation_duration = np.random.uniform(300, 450, num_samples)  # Early-stage range
        saccade_amplitude = np.random.uniform(10, 20, num_samples)    # Early-stage range
        pupil_dilation = np.random.uniform(4, 6, num_samples)         # Early-stage range
        blink_rate = np.random.uniform(15, 20, num_samples)           # Early-stage range
    else:  # category == 'alz'
        fixation_duration = np.random.uniform(450, 600, num_samples)  # Alzheimer's range
        saccade_amplitude = np.random.uniform(1, 10, num_samples)     # Alzheimer's range
        pupil_dilation = np.random.uniform(6, 8, num_samples)         # Alzheimer's range
        blink_rate = np.random.uniform(20, 25, num_samples)           # Alzheimer's range

    gaze_x = np.random.uniform(0, 1920, num_samples)              # screen resolution width
    gaze_y = np.random.uniform(0, 1080, num_samples)              # screen resolution height

    # Combine features into a single dataset
    X = np.column_stack((fixation_duration, saccade_amplitude, pupil_dilation, blink_rate, gaze_x, gaze_y))

    # Generate labels (0: Healthy, 1: Early-stage Alzheimer's, 2: Alzheimer's)
    y = 0 if category == 'healthy' else (1 if category == 'early_alz' else 2)

    return X, np.full(num_samples, y)

# Function to generate and save data for multiple individuals
def generate_and_save_data(num_individuals, num_samples_per_person, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    all_data = []

    for i in range(num_individuals):
        if i < 200:
            category = 'healthy'
        elif i < 300:
            category = 'early_alz'
        else:
            category = 'alz'

        X, y = generate_eye_tracking_data(num_samples_per_person, category)
        person_data = np.column_stack((np.full(num_samples_per_person, i), X, y))
        all_data.append(person_data)

        # Save each person's data to a CSV file
        df = pd.DataFrame(person_data, columns=['PersonID', 'FixationDuration', 'SaccadeAmplitude', 'PupilDilation', 'BlinkRate', 'GazeX', 'GazeY', 'Label'])
        df.to_csv(os.path.join(output_dir, f'person_{i}.csv'), index=False)

    # Combine all data into a single dataframe
    all_data = np.vstack(all_data)
    df_all = pd.DataFrame(all_data, columns=['PersonID', 'FixationDuration', 'SaccadeAmplitude', 'PupilDilation', 'BlinkRate', 'GazeX', 'GazeY', 'Label'])
    df_all.to_csv(os.path.join(output_dir, 'all_data.csv'), index=False)

# Generate and save data
output_dir = 'eye_tracking_data'
num_individuals = 400  # Number of individuals
num_samples_per_person = 4  # Number of samples per individual

generate_and_save_data(num_individuals, num_samples_per_person, output_dir)

# Load the data from CSV
df = pd.read_csv(os.path.join(output_dir, 'all_data.csv'))

# Split the data into features and labels
X = df[['FixationDuration', 'SaccadeAmplitude', 'PupilDilation', 'BlinkRate', 'GazeX', 'GazeY']].values
y = df['Label'].values

# Reshape X for LSTM input [samples, timesteps, features]
X = X.reshape((X.shape[0], 1, X.shape[1]))

# Split the data into 70% train, 15% validation, 15% test
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Build the Bi-LSTM Model with Attention Mechanism
def build_bilstm_attention_model(input_shape):
    inputs = layers.Input(shape=input_shape)

    # Bi-LSTM Layer
    x = layers.Bidirectional(layers.LSTM(128, return_sequences=True))(inputs)

    # Attention Mechanism
    attention = layers.Attention()([x, x])

    # Flatten the output of the attention layer
    x = layers.Flatten()(attention)

    # Dense Layers
    x = layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.001))(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(32, activation='relu', kernel_regularizer=regularizers.l2(0.001))(x)
    x = layers.Dropout(0.5)(x)

    # Output Layer
    outputs = layers.Dense(3, activation='softmax')(x)  # Multi-class classification for 3 classes

    model = models.Model(inputs, outputs)
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    return model

# Build and Train the Model
model = build_bilstm_attention_model((X_train.shape[1], X_train.shape[2]))
history = model.fit(X_train, y_train,
                    epochs=50,
                    batch_size=32,
                    validation_data=(X_val, y_val),
                    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)])

# Evaluate the Model
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {test_acc:.4f}')

# Save model predictions on the test set
predictions = model.predict(X_test)
df_predictions = pd.DataFrame(predictions, columns=['Healthy_Prob', 'Early_Alz_Prob', 'Alz_Prob'])
df_predictions['True_Label'] = y_test
df_predictions.to_csv(os.path.join(output_dir, 'test_predictions.csv'), index=False)

# Generate data for three new individuals (not part of the original 400)
new_individuals_data = []
categories = ['healthy', 'early_alz', 'alz']

for i, category in enumerate(categories):
    X_new, y_new = generate_eye_tracking_data(4, category)  # 4 samples per person
    new_individuals_data.append((X_new, y_new, category))

# Function to plot and analyze individual data
def analyze_new_individual(X_new, y_new, category, model):
    # Predict the individual's class
    X_person = X_new.reshape((X_new.shape[0], 1, X_new.shape[1]))
    person_prediction = model.predict(X_person)
    predicted_label = np.argmax(person_prediction.mean(axis=0))

    # Determine the class
    class_labels = ['Healthy', 'Early Alzheimer\'s', 'Alzheimer\'s']
    true_class = class_labels[y_new[0]]
    predicted_class = class_labels[predicted_label]

    # Generate line plots for each metric
    sns.set(style="whitegrid")
    plt.figure(figsize=(14, 8))
    plt.plot(X_new[:, 0], label='Fixation Duration', marker='o', color='b', linestyle='-', linewidth=2)
    plt.plot(X_new[:, 1], label='Saccade Amplitude', marker='s', color='g', linestyle='--', linewidth=2)
    plt.plot(X_new[:, 2], label='Pupil Dilation', marker='D', color='r', linestyle='-.', linewidth=2)
    plt.plot(X_new[:, 3], label='Blink Rate', marker='^', color='m', linestyle=':', linewidth=2)
    plt.title(f'New Individual ({category.capitalize()}) - Predicted: {predicted_class}', fontsize=16)
    plt.xlabel('Session', fontsize=14)
    plt.ylabel('Metric Value', fontsize=14)
    plt.legend(fontsize=12)
    plt.grid(True)
    plt.show()

    # Scatter plot of Pupil Dilation vs Fixation Duration
    plt.figure(figsize=(10, 6))
    sns.scatterplot(x=X_new[:, 0], y=X_new[:, 2], hue=['Session 1', 'Session 2', 'Session 3', 'Session 4'], palette='coolwarm', s=100)
    plt.title(f'Pupil Dilation vs Fixation Duration for {category.capitalize()} Individual', fontsize=16)
    plt.xlabel('Fixation Duration (ms)', fontsize=14)
    plt.ylabel('Pupil Dilation (mm)', fontsize=14)
    plt.legend(title='Session', fontsize=12)
    plt.grid(True)
    plt.show()

    # Time-Series Analysis Plot for each metric
    plt.figure(figsize=(14, 8))
    for i, metric in enumerate(['Fixation Duration', 'Saccade Amplitude', 'Pupil Dilation', 'Blink Rate']):
        plt.plot(X_new[:, i], marker='o', label=metric, linewidth=2)
    plt.title(f'Time-Series Analysis for {category.capitalize()} Individual', fontsize=16)
    plt.xlabel('Session', fontsize=14)
    plt.ylabel('Metric Value', fontsize=14)
    plt.legend(fontsize=12)
    plt.grid(True)
    plt.show()

    # Box Plot for each metric
    plt.figure(figsize=(14, 8))
    df_box = pd.DataFrame(X_new, columns=['Fixation Duration', 'Saccade Amplitude', 'Pupil Dilation', 'Blink Rate', 'Gaze X', 'Gaze Y'])
    sns.boxplot(data=df_box[['Fixation Duration', 'Saccade Amplitude', 'Pupil Dilation', 'Blink Rate']])
    plt.title(f'Box Plot of Metrics for {category.capitalize()} Individual', fontsize=16)
    plt.xlabel('Metric', fontsize=14)
    plt.ylabel('Value', fontsize=14)
    plt.grid(True)
    plt.show()

    # Radar Chart for metrics
    plt.figure(figsize=(8, 8))
    categories = ['Fixation Duration', 'Saccade Amplitude', 'Pupil Dilation', 'Blink Rate']
    values = X_new.mean(axis=0)[:4].tolist()
    values += values[:1]  # Close the radar chart loop

    angles = [n / float(len(categories)) * 2 * pi for n in range(len(categories))]
    angles += angles[:1]

    ax = plt.subplot(111, polar=True)
    plt.xticks(angles[:-1], categories, color='grey', size=12)
    ax.plot(angles, values, linewidth=2, linestyle='solid')
    ax.fill(angles, values, 'b', alpha=0.3)
    plt.title(f'Radar Chart for {category.capitalize()} Individual', size=16)
    plt.show()

    # Generate heatmap of the correlations between metrics
    corr_matrix = pd.DataFrame(X_new, columns=['Fixation Duration', 'Saccade Amplitude', 'Pupil Dilation', 'Blink Rate', 'Gaze X', 'Gaze Y']).corr()
    plt.figure(figsize=(10, 8))
    sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', cbar=True, annot_kws={"size": 12})
    plt.title(f'Correlation Heatmap for {category.capitalize()} Individual', fontsize=16)
    plt.show()

    # Generate distribution plot for each metric
    plt.figure(figsize=(14, 8))
    for i, column in enumerate(['Fixation Duration', 'Saccade Amplitude', 'Pupil Dilation', 'Blink Rate']):
        sns.kdeplot(X_new[:, i], label=column, fill=True, linewidth=2)
    plt.title(f'Distribution of Metrics for {category.capitalize()} Individual', fontsize=16)
    plt.xlabel('Metric Value', fontsize=14)
    plt.ylabel('Density', fontsize=14)
    plt.legend(fontsize=12)
    plt.grid(True)
    plt.show()

    # Print analysis report
    print(f"New Individual ({category.capitalize()}) Analysis Report")
    print(f"True Label: {true_class}")
    print(f"Predicted Label: {predicted_class}")
    print(f"Prediction Probabilities: {person_prediction.mean(axis=0)}")
    print("-----------------------------------------------------------")

    # Interpretation based on the plot and metrics
    print("Interpretation:")
    if predicted_class == 'Healthy':
        print("The eye-tracking metrics for this individual are within normal ranges, indicating no signs of Alzheimer's. The person likely does not have Alzheimer's disease based on the observed data.")
    elif predicted_class == 'Early Alzheimer\'s':
        print("The eye-tracking metrics suggest some early-stage cognitive decline. These patterns are consistent with early Alzheimer's disease, indicating that the person might be in the early stages of the condition.")
    else:  # Alzheimer's
        print("The eye-tracking metrics show significant deviations from normal ranges, with patterns consistent with Alzheimer's disease. The person likely has Alzheimer's disease based on the observed data.")
    print("-----------------------------------------------------------\n")

# Analyze and report on the three new individuals
for X_new, y_new, category in new_individuals_data:
    analyze_new_individual(X_new, y_new, category, model)
